In [9]:
import random

BASE64_TABLA = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"

def ascii_a_binario(texto):
    resultado = ""
    for c in texto:
        resultado += format(ord(c), "08b")
    return resultado

def binario_a_ascii(binario):
    texto = ""
    for i in range(0, len(binario), 8):
        byte = binario[i:i+8]
        if len(byte) == 8:
            texto += chr(int(byte, 2))
    return texto

def aplicar_xor(bin1, bin2):
    if len(bin1) != len(bin2):
        raise ValueError("Los binarios deben tener la misma longitud.")
    resultado = ""
    for b1, b2 in zip(bin1, bin2):
        if b1 == b2:
            resultado += "0"
        else:
            resultado += "1"
    return resultado

def generar_llave_ascii(longitud):
    llave = ""
    for _ in range(longitud):
        llave += chr(random.randint(32, 126))
    return llave

def expandir_llave(llave, longitud):
    return (llave * (longitud // len(llave) + 1))[:longitud]

def binario_a_base64(binario, bytes_originales):
    while len(binario) % 24 != 0:
        binario += "0"

    resultado = ""
    for i in range(0, len(binario), 6):
        bloque = binario[i:i+6]
        resultado += BASE64_TABLA[int(bloque, 2)]

    sobra = bytes_originales % 3
    if sobra == 1:
        resultado = resultado[:-2] + "=="
    elif sobra == 2:
        resultado = resultado[:-1] + "="

    return resultado


def base64_a_binario(texto):
    texto = texto.replace("=", "")
    binario = ""
    for c in texto:
        indice = BASE64_TABLA.index(c)
        binario += format(indice, "06b")
    return binario

def recortar_binario(binario, bytes_objetivo):
    return binario[:bytes_objetivo * 8]

# Llave fija (tamaño k)

def cifrar_llave_fija(mensaje, llave):
    llave_usada = expandir_llave(llave, len(mensaje))
    m_bin = ascii_a_binario(mensaje)
    k_bin = ascii_a_binario(llave_usada)
    c_bin = aplicar_xor(m_bin, k_bin)
    return binario_a_base64(c_bin, len(mensaje))

def descifrar_llave_fija(cipher, llave, longitud):
    c_bin = base64_a_binario(cipher)
    c_bin = recortar_binario(c_bin, longitud)
    llave_usada = expandir_llave(llave, longitud)
    k_bin = ascii_a_binario(llave_usada)
    m_bin = aplicar_xor(c_bin, k_bin)
    return binario_a_ascii(m_bin)


# Llave dinámica

def cifrar_llave_dinamica(mensaje):
    llave = generar_llave_ascii(len(mensaje))
    cipher = cifrar_llave_fija(mensaje, llave)
    return cipher, llave

def descifrar_llave_dinamica(cipher, llave, longitud):
    return descifrar_llave_fija(cipher, llave, longitud)


In [10]:
mensaje = input("Ingrese el mensaje: ")
llave = input("Ingrese la llave fija: ")

cipher = cifrar_llave_fija(mensaje, llave)
print("\nCipher (Base64):", cipher)

descifrado = descifrar_llave_fija(cipher, llave, len(mensaje))
print("Mensaje descifrado:", descifrado)



Cipher (Base64): CQsFDg==
Mensaje descifrado: hola


In [11]:
mensaje = input("Ingrese el mensaje: ")

cipher, llave_generada = cifrar_llave_dinamica(mensaje)

print("\nCipher (Base64):", cipher)
print("Llave dinámica generada:", llave_generada)

descifrado = descifrar_llave_dinamica(cipher, llave_generada, len(mensaje))
print("Mensaje descifrado:", descifrado)



Cipher (Base64): PSwHGQ==
Llave dinámica generada: UCkx
Mensaje descifrado: hola
